In [1]:
# embeddings model multi-qa-distilbert-cos-v1

model_name = 'multi-qa-distilbert-cos-v1'
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/home/lucapug/my-llm-zoomcamp/hw3/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [3]:
user_question_embed = embedding_model.encode(user_question)

In [4]:
len(user_question_embed)

768

In [5]:
user_question_embed[0:5]

array([ 0.07822262, -0.04013116,  0.03861362, -0.00017896,  0.08923467],
      dtype=float32)

### ans.1 0.07

In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [8]:
hw3_documents =[]

for doc in documents:
    if doc['course']=='machine-learning-zoomcamp':
        hw3_documents.append(doc)

len(hw3_documents)

375

In [10]:
embeddings = []

for doc in hw3_documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    emb = embedding_model.encode(qa_text)
    embeddings.append(emb)


In [12]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

### ans.2 (375, 768)

In [13]:
X[1].dot(X[1])

1.0000001

In [14]:
v = user_question_embed

scores = X.dot(v)
max(scores)

0.6506573

### ans.3 0.65